In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
df=pd.read_csv('/content/drive/MyDrive/Dataset_Churn/Churn_Modelling.csv')

# Data Cleansing

## A. Handle missing values : Wahidah

In [28]:
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1782,1783,15642002,Hayward,554,France,Female,35,6,117707.18,2,0,0,95277.15,1
976,977,15570060,Palerma,586,France,Female,43,8,132558.26,1,1,0,67046.83,1
2407,2408,15579130,Chidiegwu,708,Germany,Female,43,0,118994.84,1,1,0,181499.77,1
303,304,15624188,Chiu,712,France,Female,33,6,0.00,2,1,1,190686.16,0
1780,1781,15601008,Stevenson,802,France,Male,33,8,0.00,2,1,0,143706.18,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Dari data Churn Modelling ternyata semua data sudah bersih dan bebas dari missing value, data sudah memiliki nilai yang lengkap pada setiap kolomnya hal tersebut dipertegas dengan menggunakan syntax df.isna().sum() dengan output semua kolom bernilai 0 yang menandakan tidak ada missing value.

### B. Handle Duplicated Data : Joel

In [ ]:
# Membuat mask duplikat
duplicates = df.duplicated()

# Menampilkan baris yang merupakan duplikat
duplicate_rows = df[duplicates]
print("Baris duplikat:")
print(duplicate_rows)

Baris duplikat:
Empty DataFrame
Columns: [RowNumber, CustomerId, Surname, CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited]
Index: []


**Interpretasi:**
Pada Output di atas dapat dilihat bahwa Dataset tersebut tidak memiliki baris duplikat. Keterangan output "Empty DataFrame" menunjukkan bahwa tidak ada baris yang dianggap duplikat dalam DataFrame tersebut. Dengan demikian, tidak ada data duplikat dalam DataFrame yang diperiksa. Maka Tim dapat melanjutkan dengan analisis data tanpa perlu mengatasi adanya duplikasi pada saat ini.

## C. Handle outliers : Bazil


In [43]:
from scipy import stats

In [44]:
# Hitung Z-Score untuk fitur 'Age' dan 'CreditScore'
z_scores = np.abs(stats.zscore(df[['Age', 'CreditScore']]))
# Tentukan threshold Z-Score
threshold = 3
# Identifikasi outlier berdasarkan ambang batas Z-Score
outliers = df[(z_scores > threshold).any(axis=1)]
# Tangani outlier dengan menghapusnya
df = df[(z_scores <= threshold).all(axis=1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9859 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        9859 non-null   int64  
 1   CustomerId       9859 non-null   int64  
 2   Surname          9859 non-null   object 
 3   CreditScore      9859 non-null   int64  
 4   Geography        9859 non-null   object 
 5   Gender           9859 non-null   object 
 6   Age              9859 non-null   int64  
 7   Tenure           9859 non-null   int64  
 8   Balance          9859 non-null   float64
 9   NumOfProducts    9859 non-null   int64  
 10  HasCrCard        9859 non-null   int64  
 11  IsActiveMember   9859 non-null   int64  
 12  EstimatedSalary  9859 non-null   float64
 13  Exited           9859 non-null   int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Feature Age, dan feature CreditScore merupakan feature yang memiliki outlier, serta distribusinya secara berturut-turut adalah positvely skewed, dan Negatively skewed. Maka diperlukan penanganan atas outlier tersebut dengan menggunakan metode Z-Score. Jumlah data setelah dihapusnya outlier tersebut berjumlah 9859, sehingga terdapat 141 nilai ekstrem yang terdapat pada feature Age, dan CreditScore.

## D. Feature transformation : Ummi


In [37]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

# split data menjadi data train dan data test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [38]:
#proses scalling kolom CreditScore
scaller = MinMaxScaler()
scaller.fit (df_train['CreditScore'].values.reshape(len(df_train),1))
df_train['CreditScore_norm'] = scaller.transform(df_train['CreditScore'].values.reshape(len(df_train), 1))
df_test['CreditScore_norm'] = scaller.transform(df_test['CreditScore'].values.reshape(len(df_test), 1))

#proses scalling kolom Age
scaller = MinMaxScaler()
scaller.fit (df_train['Age'].values.reshape(len(df_train),1))
df_train['Age_norm'] = scaller.transform(df_train['Age'].values.reshape(len(df_train), 1))
df_test['Age_norm'] = scaller.transform(df_test['Age'].values.reshape(len(df_test), 1))

#proses scalling kolom Balance
scaller = MinMaxScaler()
scaller.fit (df_train['Balance'].values.reshape(len(df_train),1))
df_train['Balance_norm'] = scaller.transform(df_train['Balance'].values.reshape(len(df_train), 1))
df_test['Balance_norm'] = scaller.transform(df_test['Balance'].values.reshape(len(df_test), 1))

#proses scalling kolom EstimatedSalary
scaller = MinMaxScaler()
scaller.fit (df_train['EstimatedSalary'].values.reshape(len(df_train),1))
df_train['EstimatedSalary_norm'] = scaller.transform(df_train['EstimatedSalary'].values.reshape(len(df_train), 1))
df_test['EstimatedSalary_norm'] = scaller.transform(df_test['EstimatedSalary'].values.reshape(len(df_test), 1))

In [39]:
df_train[['CreditScore','CreditScore_norm','Age','Age_norm','Balance','Balance_norm','EstimatedSalary','EstimatedSalary_norm']].describe()

,CreditScore,CreditScore_norm,Age,Age_norm,Balance,Balance_norm,EstimatedSalary,EstimatedSalary_norm
count,7887.000000,7887.000000,7887.000000,7887.000000,7887.000000,7887.000000,7887.000000,7887.000000
mean,650.091036,0.589509,38.403702,0.392379,77002.236960,0.306906,100354.931928,0.501569
std,96.196497,0.197529,9.606027,0.184731,62403.836887,0.248722,57427.220562,0.287276
min,363.000000,0.000000,18.000000,0.000000,0.000000,0.000000,90.070000,0.000000
25%,583.000000,0.451745,32.000000,0.269231,0.000000,0.000000,51130.245000,0.255325
50%,651.000000,0.591376,37.000000,0.365385,97473.870000,0.388500,101108.850000,0.505340
75%,717.000000,0.726899,43.000000,0.480769,128022.775000,0.510258,149085.630000,0.745341
max,850.000000,1.000000,70.000000,1.000000,250898.090000,1.000000,199992.480000,1.000000


In [40]:
df_test[['CreditScore','CreditScore_norm','Age','Age_norm','Balance','Balance_norm','EstimatedSalary','EstimatedSalary_norm']].describe()

,CreditScore,CreditScore_norm,Age,Age_norm,Balance,Balance_norm,EstimatedSalary,EstimatedSalary_norm
count,1972.000000,1972.000000,1972.000000,1972.000000,1972.000000,1972.000000,1972.000000,1972.000000
mean,652.660243,0.594785,38.529412,0.394796,74879.868519,0.298447,98871.283707,0.494147
std,97.042659,0.199266,9.941567,0.191184,62323.468866,0.248402,57761.618889,0.288949
min,410.000000,0.096509,18.000000,0.000000,0.000000,0.000000,11.580000,-0.000393
25%,584.750000,0.455339,32.000000,0.269231,0.000000,0.000000,50202.460000,0.250684
50%,654.500000,0.598563,37.000000,0.365385,95601.810000,0.381038,95517.350000,0.477369
75%,721.000000,0.735113,43.000000,0.480769,125874.987500,0.501698,150251.060000,0.751171
max,850.000000,1.000000,70.000000,1.000000,216109.880000,0.861345,199970.740000,0.999891


Pada proses future transformation terlebih dahulu dilakukan proses split data menjadi data train dan data test. Kemudian dilakukan proses future transformation dengan metode normalisasi yang mana akan menghasilkan nilai minimal 0 dan nilai maksimal 1. Normalisasi dipilih karena distribusi data sudah mendekati distribusi normal.

> Indented block



## E. Feature encoding : Rafif


In [45]:
# Soal 1 Data Cleansing (Feature encoding)
mapping_Gender = {
    'Female': 0,
    'Male': 1
}
df['Gender'] = df['Gender'].map(mapping_Gender)
data1 = pd.get_dummies(df['Geography'],prefix ='Geography')

datanew = pd.concat([df, data1], axis=1)

display(datanew)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,1,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,France,1,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,France,0,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1,0,1,0


Keterangan Pemilihan teknik feature encoding yang paling cocok untuk kolom "Gender" dan "Geography"

Label Encoding: Label encoding cocok untuk kolom "Gender" karena kolom ini hanya memiliki dua nilai kategori yaitu "Male" dan "Female".

One-Hot Encoding: One-hot encoding lebih sesuai untuk kolom "Geography" karena kolom ini memiliki lebih dari dua kategori yang mungkin. Dalam one-hot encoding, setiap kategori diubah menjadi kolom baru, dan setiap baris akan memiliki nilai 1 atau 0, tergantung pada kategori yang cocok. Ini memungkinkan model untuk mengatasi variasi yang lebih besar dalam kolom "Geography."

## F. Handle class imbalance : Alisia

In [46]:
# Pemisahan Features dan Target
x = datanew.drop(['Surname', 'Exited'], axis=1)
y = datanew['Exited']

print(x.shape)
print(y.shape)

(9859, 15)
(9859,)


*   X merupakan Features yang mana berisikan kolom-kolom yang akan diperhitungkan dalam permodelan machine learning
*   Y merupakan Target dari permodelan machine learning yang akan dibuat, yang berisikan kolom Exited karena kita akan memprediksi churn nasabah

In [22]:
# Penentuan perbandingan kelas Exited untuk melihat kesenjangan antar kelas(Class Imbalance) dari jumlah presentasenya

class_count_0, class_count_1 = datanew['Exited'].value_counts()

total_data = class_count_0 + class_count_1
minority_ratio = round((class_count_1 / total_data) * 100, 2)
majority_ratio = round((class_count_0 / total_data) * 100, 2)

print('Presentase Kelas Minoritas:', minority_ratio, '%')
print('Presentase Kelas Mayoritas:', majority_ratio, '%')

Presentase Kelas Minoritas: 20.47 %
Presentase Kelas Mayoritas: 79.53 %


Dari hasil perhitungan presentase antar kelas, yang mana menunjukkan bahwa kelas ada di derajat Moderate (1 - 20 %), sehingga untuk mengembangkan permodelan macine learning yang lebih baik, kedua kelas harus diseimbangkan

In [16]:
# Mengatasi kesenjangan antar kelas dengan oversampling SMOTE

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

x_over, y_over = over_sampling.RandomOverSampler(sampling_strategy=0.5).fit_resample(x, y)
x_over_SMOTE, y_over_SMOTE = SMOTE(sampling_strategy=0.5).fit_resample(x, y)

print(pd.Series(y_over).value_counts())

0    7841
1    3920
Name: Exited, dtype: int64


# Feature Engineering

## A. Feature selection (membuang feature yang kurang relevan atau redundan) : Sari

In [47]:
# Melakukan feature selection
x = datanew.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Geography','Exited'])
y = datanew['Exited']


Melakukan featue selection (membuang feature yang kurang relevan atau redundan) pada beberepa kolom yaitu 'RowNumber', 'CustomerId', 'Surname', 'Geography' dan 'Exited'. Karena dirasa kurang relevan dengan target yaitu 'Exited'. Dan menghapus kolom 'Exited' karena sudah menjadi target itu sendiri.


## B. Feature extraction (membuat feature baru dari feature yang sudah ada) : Jungly

In [48]:
#Menambahkan feature baru
x['Balance-to-SallaryRatio'] = x.Balance/x.EstimatedSalary
x['Tenure-to-AgeRatio'] = x.Tenure/x.Age
x['Age-to-CreditRatio'] = x.Age/x.CreditScore

In [49]:
x.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Balance-to-SallaryRatio,Tenure-to-AgeRatio,Age-to-CreditRatio
9363,634,1,35,3,0.00,2,1,1,19515.48,0,0,1,0.000000,0.085714,0.055205
8131,613,1,33,3,155736.42,2,1,1,57751.21,0,1,0,2.696678,0.090909,0.053834
2664,586,1,29,3,0.00,2,1,1,142238.54,1,0,0,0.000000,0.103448,0.049488
7450,712,0,39,5,163097.55,2,1,1,23702.42,0,0,1,6.881051,0.128205,0.054775
2128,743,1,37,8,69143.91,2,0,1,105780.18,1,0,0,0.653657,0.216216,0.049798


## C. Tuliskan minimal 4 feature tambahan (selain yang sudah tersedia di dataset) yang mungkin akan sangat membantu membuat performansi model semakin bagus (ini hanya ide saja, untuk menguji kreativitas teman-teman, tidak perlu benar-benar dicari datanya dan tidak perlu diimplementasikan) : Jungly

1. Feature Partner yaitu feature untuk melihat apakah nasabah memiliki pasangan atau tidak karena bisa jadi jika nasabah memiliki partner dapat tidak churn
2. Feature total tanggungan yaitu feature untuk melihat tanggungan yang menjadi tanggung jawab nasabah, sehingga dapat diketahui potensi churn berdasarkan nasabah yang memiliki tanggungan berapa saja dan produk apa saja yang cocok untuk nasabah yang memiliki tanggungan.
3. Feature Profesi yaitu feature untuk melihat apa profesi atau pekerjaan dari nasabah sehingga dapat menyesuaikan produk yang diberikan kepada nasabah agar tidak churn.
4. Feature masa aktif nasabah yaitu feature untuk melihat lama waktu dari seorang nasabah berada di bank, baik dari awal masuk atau sampai keluar.